<a href="https://colab.research.google.com/github/weso500/QMLIIOT/blob/main/TEPIsolation_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================
# Quantum Random Features + Isolation Forest (semi-supervised)
# =========================
!pip -q install qiskit==1.2.4 scikit-learn==1.6.1 numpy pandas

import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, precision_recall_fscore_support
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# ---- Tunables ----
N_FEATS = 8          # use 6–8 to keep circuits small; try 11 if time allows
N_FEATURE_MAPS = 256 # number of random quantum features (128–512)
DEPTH = 2            # random circuit depth (2–3)
RANDOM_SEED = 42

# 1) Load
train_df = pd.read_csv('/content/drive/MyDrive/QMLIIOT/TEP9Train_large.csv')  # normals only
test_df  = pd.read_csv('/content/drive/MyDrive/QMLIIOT/TEP9Test_large.csv')
Xn_raw = train_df.to_numpy().astype(np.float32)
Xt_raw = test_df.to_numpy().astype(np.float32)
y_test = np.concatenate((np.zeros(500), np.ones(25))).astype(int)

# 2) Scale on normals, simple top-variance feature selection (no PCA)
scaler = StandardScaler().fit(Xn_raw)
Xn = scaler.transform(Xn_raw)
Xt = scaler.transform(Xt_raw)
vars_ = Xn.var(axis=0)
keep = np.argsort(vars_)[::-1][:N_FEATS]
Xn, Xt = Xn[:, keep], Xt[:, keep]

rng = np.random.default_rng(RANDOM_SEED)

# 3) Build random quantum feature functions φ_j(x)
def build_random_circuit(n_qubits, depth, rng):
    qc = QuantumCircuit(n_qubits)
    for _ in range(depth):
        # random single-qubit rotations
        for q in range(n_qubits):
            a, b, c = rng.uniform(0, 2*np.pi, size=3)
            qc.ry(a, q); qc.rz(b, q); qc.rx(c, q)
        # random entanglement (linear chain)
        for q in range(n_qubits-1):
            qc.cx(q, q+1)
    return qc

# Pre-generate random circuits
circuits = [build_random_circuit(N_FEATS, DEPTH, rng) for _ in range(N_FEATURE_MAPS)]

# Angle-embed x into the circuit by appending Ry(pi/2 * x_i) to each wire before random block
def embed_and_eval(state, x, base_qc):
    n = len(x)
    prep = QuantumCircuit(n)
    for i in range(n):
        prep.ry(float(np.pi/2 * x[i]), i)
    full = prep.compose(base_qc)
    sv = Statevector.from_instruction(full)
    # A simple set of observables: Z expectation on each qubit; sum-pool to one scalar
    # You can also pick a subset or random Pauli strings; we keep it simple and fast here
    probs = np.abs(sv.data)**2
    expZ = []
    for q in range(n):
        mask = 1 << (n-1-q)
        exp = 0.0
        for idx, p in enumerate(probs):
            bit = (idx & mask) >> (n-1-q)
            exp += (1 - 2*bit) * p   # +1 for |0>, -1 for |1>
        expZ.append(exp)
    # return both mean and a few coordinates for richer features
    return np.array([np.mean(expZ), np.std(expZ)], dtype=np.float32)

def quantum_random_features(X):
    # shape: (n_samples, 2 * N_FEATURE_MAPS)
    out = np.zeros((X.shape[0], 2*N_FEATURE_MAPS), dtype=np.float32)
    state = None
    for j, qc in enumerate(circuits):
        phi = np.vstack([embed_and_eval(state, x, qc) for x in X])
        out[:, 2*j:2*j+2] = phi
    return out

# 4) Build features
Zn = quantum_random_features(Xn)   # normals
Zt = quantum_random_features(Xt)   # test

# 5) Train Isolation Forest on normals only
iso = IsolationForest(
    n_estimators=300, max_samples='auto', contamination='auto',
    random_state=RANDOM_SEED, n_jobs=-1
).fit(Zn)

# Scores (higher = more anomalous): flip sign from sklearn’s convention
scores = -iso.score_samples(Zt)

# 6) Evaluate
auc = roc_auc_score(y_test, scores)
print(f"ROC-AUC: {auc:.4f}")

# Pick operating threshold by Youden J on ROC
fpr, tpr, thr = roc_curve(y_test, scores)
j = np.argmax(tpr - fpr); thr_star = thr[j] if j < len(thr) else np.median(scores)
y_pred = (scores >= thr_star).astype(int)
cm = confusion_matrix(y_test, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary', zero_division=0)
print("Confusion matrix [[TN, FP],[FN, TP]]:\n", cm)
print(f"Precision={prec:.3f}  Recall={rec:.3f}  F1={f1:.3f}  Thr={thr_star:.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 17.2 MB/s eta 0:00:00
ROC-AUC: 0.5517
Confusion matrix [[TN, FP],[FN, TP]]:
 [[179 321]
 [  3  22]]
Precision=0.064  Recall=0.880  F1=0.120  Thr=0.4424
